# Grid rotation


In [ ]:
import pandas as pd
import matplotlib
import nlmod
import logging

# set the log-level to INFO, so more information is shown
# (compared to the default setting of WARNING)
logging.basicConfig(level=logging.INFO)

## Generate a model Dataset
We generate a model dataset with a rotation of 10 degrees counterclockwise.

In [ ]:
ds = nlmod.mlayers.get_default_ds(
    [0, 1000, 0, 1000],
    angrot=10.0,
    xorigin=200000,
    yorigin=500000,
    delr=10.0,
    model_name="nlmod",
    model_ws="model11",
)

ds = nlmod.mdims.set_ds_time(ds, time=pd.date_range("2015", "2022", freq="MS"))

## Use a disv-grid
or not, by commenting out the line below

In [ ]:
ds = nlmod.mgrid.refine(ds)

## Add AHN
Download the ahn, resample to the new grid (using the method 'average') and compare.

In [ ]:
# Dwonload AHN
extent = nlmod.resample.get_extent(ds)
ahn = nlmod.read.ahn.get_ahn_within_extent(extent)

# Resample to the grid
ds["ahn"] = nlmod.resample.structured_da_to_ds(ahn, ds, method="average")

# Compare original ahn to the resampled one
f, axes = nlmod.plot.get_map(extent, ncols=2)
norm = matplotlib.colors.Normalize()
pc = nlmod.plot.da(ahn, ax=axes[0], norm=norm)
nlmod.plot.colorbar_inside(pc, ax=axes[0])
pc = nlmod.plot.da(ds["ahn"], ds=ds, ax=axes[1], rotated=True, norm=norm)
nlmod.plot.colorbar_inside(pc, ax=axes[1])

## Download surface water
Download BGT-polygon data, add stage information from the waterboar, and grid the polygons. Because we use a rotated grid, the bgt-polygons are in model coordinates.

In [ ]:
bgt = nlmod.gwf.surface_water.get_gdf(ds)

In [ ]:
bgt.plot()

## Download knmi-data

In [ ]:
knmi_ds = nlmod.read.knmi.get_recharge(ds)
ds.update(knmi_ds)

## Generate flopy-model
A simulation and groundwater flow model, with some standard packages

In [ ]:
# %%
# create simulation
sim = nlmod.gwf.sim(ds)

# create time discretisation
tdis = nlmod.gwf.tdis(ds, sim)

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# create ims
ims = nlmod.gwf.ims(sim, complexity="complex")

# Create discretization
nlmod.gwf.dis(ds, gwf)

# create node property flow
nlmod.gwf.npf(ds, gwf, save_flows=True)

# Create the initial conditions package
nlmod.gwf.ic(ds, gwf, starting_head=0.0)

# Create the output control package
nlmod.gwf.oc(ds, gwf)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

# create storage package
sto = nlmod.gwf.sto(ds, gwf)

## Add surface water
To the groundwater flow model

In [ ]:
nlmod.gwf.gdf_to_seasonal_pkg(bgt, gwf, ds)

## Run the model and read the heads

In [ ]:
# run the model
nlmod.gwf.write_and_run_model(gwf, ds)

# read the heads
head = nlmod.util.get_heads_dataarray(ds)

# Plot the heads in layer 1
When grid rotation is used, nlmod.plot.da() plots a DataArray in model coordinates. If we want to plot in realworld coordinates, we set the optional parameter 'rotated' to True.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
pc = nlmod.plot.da(head.sel(layer=1).mean("time"), ds=ds, edgecolor="k", rotated=True)
cbar = nlmod.plot.colorbar_inside(pc)
for label in cbar.ax.yaxis.get_ticklabels():
    label.set_bbox(dict(facecolor="w", alpha=0.5))
bgt.plot(ax=ax, edgecolor="k", facecolor="none")